<a href="https://colab.research.google.com/github/Kobeieii/Project/blob/main/The_Movies_Dataset_Recoomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Kaggle dataset into Google Colab

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 4.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=75f0f382ec2ce33c757eec5e5925b67bae70fa73db78a451913e7d4a36b5d700
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"
%cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

 95% 217M/228M [00:05<00:00, 60.4MB/s]
100% 228M/228M [00:05<00:00, 44.8MB/s]


In [ ]:
!unzip "/content/gdrive/MyDrive/Kaggle/the-movies-dataset.zip" -d "/content/gdrive/MyDrive/Kaggle/movies_dataset"

Archive:  /content/gdrive/MyDrive/Kaggle/the-movies-dataset.zip
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/credits.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/keywords.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/links.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/links_small.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/movies_metadata.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/ratings.csv  
  inflating: /content/gdrive/MyDrive/Kaggle/movies_dataset/ratings_small.csv  


# Read data

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import warnings; warnings.simplefilter('ignore')

In [ ]:
#movies_metadata.csv: The main Movies Metadata file. 

movie_df = pd.read_csv('/content/gdrive/MyDrive/Kaggle/movies_dataset/movies_metadata.csv')
movie_df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

links_small = pd.read_csv('/content/gdrive/MyDrive/Kaggle/movies_dataset/links_small.csv')
links_small.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [ ]:
#Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

credits = pd.read_csv('/content/gdrive/MyDrive/Kaggle/movies_dataset/credits.csv')
credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [ ]:
# Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

keywords = pd.read_csv('/content/gdrive/MyDrive/Kaggle/movies_dataset/keywords.csv')
keywords.head(2)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


#Data preparation

In [ ]:
#format data types
movie_df = movie_df.drop([19730, 29503, 35587])
movie_df['id'] = movie_df['id'].astype('int')
credits['id'] = credits['id'].astype('int')

links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
md = movie_df.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 27)

In [ ]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int64  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev



*   cast: เลือกนักแสดงนำ 3 คนแรก
*   crew: เลือกเฉพาะผู้กำกับ
*genres: เลือกเฉพาะประเภทของหนัง


In [ ]:
#format string to list
smd['cast'] = smd['cast'].apply(eval)
smd['crew'] = smd['crew'].apply(eval)
smd['genres'] = smd['genres'].apply(eval)
smd['keywords'] = smd['keywords'].apply(eval)

#find director & lead actor
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x])
smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x])
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",John Lasseter
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",Howard Deutch
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",Charles Shyer


In [ ]:
print(smd['cast'][0])
print(smd['director'][0])

['Tom Hanks', 'Tim Allen', 'Don Rickles']
John Lasseter


In [ ]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...","[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...","[joejohnston, joejohnston, joejohnston]"


In [ ]:
smd['release_date'] = pd.to_datetime(smd['release_date'],format='%Y-%m-%d', errors='coerce')
smd['year'] = smd['release_date'].dt.year

In [ ]:
# บางคำที่ความหมายแบบเดียวกัน แต่สะกดต่างออกไป จึงต้องใช้ SnowballStemmer
stemmer = SnowballStemmer('english')
stemmer.stem('jealousy')

'jealousi'

In [ ]:
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))
smd['soup']

0        jealousi toy boy friendship friend rivalri boy...
1        boardgam disappear basedonchildren'sbook newho...
2        fish bestfriend duringcreditsst oldmen walterm...
3        basedonnovel interracialrelationship singlemot...
4        babi midlifecrisi confid age daughter motherda...
                               ...                        
40952    friendship brickmak sidneypoitier wendycrewson...
41172    bollywood akshaykumar ileanad'cruz eshagupta t...
41225    bollywood hrithikroshan poojahegde kabirbedi a...
41391    monster godzilla giantmonst destruct kaiju toy...
41669    music documentari paulmccartney ringostarr joh...
Name: soup, Length: 9219, dtype: object

#Content based Filtering

Content-based Filtering จะดูที่ลักษณะของสินค้าที่จะแนะนำ และแนะนำสิ่งที่มีลักษณะหรือมีคำอธิบายคล้ายกับโปรไฟล์ของผู้ใช้ รวมถึงลักษณะของสิ่งที่ผู้ใช้เคยใช้หรือเคยชอบ เช่น ระบบจะแนะนำหนังที่เนื้อหาของหนังมีความคล้ายกับหนังที่ผู้ใช้เคยดูมาก่อนหน้านี้ ดังนั้นการใช้ Content-based Filtering จะต้องมีข้อมูลคุณลักษณะของสินค้า เช่น กลุ่ม/ประเภท ขนาด ราคา สี สไตล์ ฟังก์ชั่น ตำแหน่งแบรนด์ดิ้ง หรือคำอธิบายตัวสินค้าแบบย่อ เป็นต้น

ข้อดี


1.   เนื่องจากระบบแนะนำดูโปรไฟล์ของผู้ใช้แต่ละคนแยกออกจากกัน สินค้าที่แนะนำจะค่อนข้างตรงกับรสนิยมของผู้ใช้ที่มีรสนิยมแตกต่างจากคนส่วนใหญ่
2.   ไม่ต้องการข้อมูลจากลูกค้าอื่น หากมีสินค้าใหม่เข้ามา ระบบสามารถแนะนำสินค้าได้เลย

ข้อเสีย


1.   การเตรียมแคตาล็อคข้อมูลสินค้าซึ่งใช้เวลามาก และการสร้าง feature ที่เหมาะสมเพื่ออธิบายตัวสินค้าและโปรไฟล์ของผู้ใช้ซึ่งขึ้นอยู่กับแนวของสินค้า
2.   สินค้าที่แนะนำจะไม่ค่อยมีความหลากหลายมากนัก





In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
cosine_sim[0]

array([1.        , 0.02441931, 0.02654659, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

ระบบ recommendation system นี้จะแนะนำหนังโดยเรียงลำดับจาก

*   ประเภทของหนังที่คล้ายกันที่สุด
*   มีนักแสดงนำที่ใกล้เคียงกันที่สุด
*ผู้กำกับคนเดียวกัน
*เรียงลำดับตามเรทติ้งที่ผ่านการ weight มาแล้ว



In [ ]:
def recommendations(title):
    #หา idx ที่ soup คล้ายกันที่สุด 26 อันดับแรก
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    #sort ตาม weighted rating ของแต่ละเรื่อง
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = weighted_rating(qualified, m, C)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
recommendations('Inception')

,title,vote_count,vote_average,year,wr
6981,The Dark Knight,12269,8,2008,7.691956
8613,Interstellar,11187,8,2014,7.667324
6623,The Prestige,4510,8,2006,7.343268
3381,Memento,4168,8,2000,7.308781
8031,The Dark Knight Rises,9263,7,2012,6.814171
6218,Batman Begins,7511,7,2005,6.781136
4173,Minority Report,2663,7,2002,6.569248
7286,X-Men Origins: Wolverine,4086,6,2009,6.029168
8207,Looper,4777,6,2012,6.026338
7903,Green Lantern,2551,5,2011,5.517233
